In [14]:
import torch
from torchvision import transforms, datasets
import os
import cv2
import numpy as np
import pandas as pd
import torch.nn as nn
from torch import optim as optim
from tqdm import tqdm
from torch.utils.data import DataLoader,TensorDataset

In [15]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [16]:
DataDir = "/Users/test/Desktop/Master_Thesis_Flow/real-nvp/data/GPR_Data/B200/B200 Pictures HR"

In [11]:
def CropTrainingData(DataPath):
    
    # Storing Images
    Cropped = []
    
    # Iteration over all0 Image Files
    for Img in tqdm(os.listdir(DataPath)):
        # Reading File Path with Error Handling
        try:
            ImgArray = cv2.imread(os.path.join(DataPath,Img),cv2.IMREAD_GRAYSCALE)
            # Append Image Snippets to Storing Variable
            if ImgArray.shape == (128,128):
                Cropped.append(ImgArray[0:128,0:128])
                Cropped.append(ImgArray[0:128,124:252])
                Cropped.append(ImgArray[0:128,248:376])
                Cropped.append(ImgArray[0:128,372:500])
        except:
            pass
    # convert to Numpy Array and add Dimension like in PyTorch Datasets        
    return np.expand_dims(np.array(Cropped),axis=1)

In [12]:
TrainingData = CropTrainingData(DataDir)

100%|██████████| 608/608 [00:01<00:00, 361.10it/s]


In [17]:
GPRData = torch.Tensor(TrainingData)
Targets = torch.Tensor().new_ones((len(TrainingData),1))

GPRDataset = TensorDataset(GPRData, Targets)

In [18]:
print(len(TrainingData))

0


In [8]:
training_parameters = {
    "n_epochs": 100,
    "batch_size": 100,
}

In [20]:
# HyperParameters
Epochs = 200
BatchSize = 128
LatentDim = 100


# Load Training Data
TrainData = DataLoader(GPRDataset, BatchSize, shuffle = True, drop_last=True)

In [21]:
class Discriminator(nn.Module):
    def __init__(self):
        super().__init__()
        
        self.label_emb = nn.Embedding(10, 10)
        
        self.model = nn.Sequential(
            nn.Linear(794, 1024),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Dropout(0.3),
            nn.Linear(1024, 512),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Dropout(0.3),
            nn.Linear(512, 256),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Dropout(0.3),
            nn.Linear(256, 1),
            nn.Sigmoid()
        )
    
    def forward(self, x, labels):
        x = x.view(x.size(0), 784)
        c = self.label_emb(labels)
        x = torch.cat([x, c], 1)
        out = self.model(x)
        return out.squeeze()

In [22]:
class Generator(nn.Module):
    def __init__(self):
        super().__init__()
        
        self.label_emb = nn.Embedding(10, 10)
        
        self.model = nn.Sequential(
            nn.Linear(110, 256),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(256, 512),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(512, 1024),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(1024, 784),
            nn.Tanh()
        )
    
    def forward(self, z, labels):
        z = z.view(z.size(0), 100)
        c = self.label_emb(labels)
        x = torch.cat([z, c], 1)
        out = self.model(x)
        return out.view(x.size(0), 28, 28)

In [23]:
generator = Generator().cuda()
discriminator = Discriminator().cuda()

RuntimeError: Cannot initialize CUDA without ATen_cuda library. PyTorch splits its backend into two shared libraries: a CPU library and a CUDA library; this error has occurred because you are trying to use some CUDA functionality, but the CUDA library has not been loaded by the dynamic linker for some reason.  The CUDA library MUST be loaded, EVEN IF you don't directly use any symbols from the CUDA library! One common culprit is a lack of -Wl,--no-as-needed in your link arguments; many dynamic linkers will delete dynamic library dependencies if you don't depend on any of their symbols.  You can check if this has occurred by using ldd on your binary to see if there is a dependency on *_cuda.so library.

In [9]:
class GPRDataset(Dataset):
    def __init__ (self, transform=None):
        #self.root_dir = root_dir
        self.transform = transform
        #self.img_file = img_file
        pictures_df = pd.read_csv('/Users/test/Desktop/Master_Thesis_Flow/real-nvp/data/GPR_Data/B200/B200 Pictures HR/labels_complete_dataset_607.csv',  sep='\n')
        pictures_df.columns =  ['filename', 'width', 'height','class','xmin', 'ymin','xmax','ymax']
        self.labels = pictures_df.label.values
        self.images = pictures_df.iloc[:, 1:].values.astype('uint8').reshape(-1, 128, 128)
        print(pictures_df.columns)    
    def __len__(self):
        
        return len(self.images)
    
    def __getitem__(self, idx):
        
            
        img_name = os.path.join(self.root_dir, self.img_file[idx])
        #image = io.imread(img_name)
        image = Image.fromarray(self.images[idx])
        label = self.labels[idx]
         #image = image[0:128,0:128]
            
        return image, label 

#if __name__ == '__main__':
  #  from torch.utils.data import DataLoader
   # dataset = GPRDataset()
    #dataloader = DataLoader(dataset, batch_size=50, shuffle=True)
   # for i, batch in enumerate(dataloader):
     #   print(i, batch)

In [10]:
dataset = GPRDataset()

ValueError: Length mismatch: Expected axis has 1 elements, new values have 8 elements